# Running an Individual Test Suite

This notebook shows how to run an individual test suite.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install --upgrade validmind

Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

2023-07-10 14:47:20,927 - INFO - api_client - Connected to ValidMind. Project: Customer Churn Model - Test for Notebooks, Sydney (cljxccag7000016mkriy2l77l)


In [3]:
%matplotlib inline

import xgboost as xgb

## Load the Demo Dataset

In [4]:
# You can also import taiwan_credit like this:
# from validmind.datasets.classification import taiwan_credit as demo_dataset
from validmind.datasets.classification import customer_churn as demo_dataset

df = demo_dataset.load_data()

In [5]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

2023-07-10 14:47:31,244 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 14:47:31,269 - INFO - dataset - Inferring dataset types...


## List available test suites

In [6]:
vm.test_suites.list_suites()

ID,Name,Description,Test Plans
binary_classifier_full_suite,BinaryClassifierFullSuite,Full test suite for binary classification models.,"tabular_dataset_description, tabular_data_quality, binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
binary_classifier_model_validation,BinaryClassifierModelValidation,Test suite for binary classification models.,"binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
tabular_dataset,TabularDataset,Test suite for tabular datasets.,"tabular_dataset_description, tabular_data_quality"
time_series_dataset,TimeSeriesDataset,Test suite for time series datasets.,"time_series_data_quality, time_series_univariate, time_series_multivariate"
time_series_model_validation,TimeSeriesModelValidation,Test suite for time series model validation.,"regression_model_description, regression_models_evaluation, time_series_forecast, time_series_sensitivity"


## Run the Data Validation Test suite

In [7]:
tabular_suite = vm.run_test_suite("tabular_dataset", dataset=vm_dataset)

2023-07-10 14:47:44,674 - ERROR - api_client - Error logging dataset to ValidMind API
2023-07-10 14:47:45,452 - ERROR - test_plan - Failed to log result: TestPlanDatasetResult for test plan result 'TestPlanDatasetResult'


ClientConnectorCertificateError: Cannot connect to host api.dev.vm.validmind.ai:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')]

## Run the Model Validation Test suite

We will need to preprocess the dataset and produce the training, test and validation splits first.

### Prepocess the Raw Dataset

In [8]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

### Train a Model for Testing

We train a simple customer churn model for our test.

In [9]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

## Import and Run the Individual Test Suite

### Initialize ValidMind objects

We initize the objects required to run test plans using the ValidMind framework 

In [10]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

2023-07-10 14:49:39,031 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 14:49:39,036 - INFO - dataset - Inferring dataset types...
2023-07-10 14:49:39,211 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 14:49:39,212 - INFO - dataset - Inferring dataset types...


### Run the Binary Classification Test Suite

In [11]:
model_suite = vm.run_test_suite("binary_classifier_model_validation", model=vm_model)

2023-07-10 14:49:51,659 - ERROR - api_client - Error logging metrics to ValidMind API
2023-07-10 14:49:51,672 - ERROR - test_plan - Failed to log result: TestPlanMetricResult(result_id="model_metadata", metric, figures) for test plan result 'TestPlanMetricResult(result_id="model_metadata", metric, figures)'


ClientConnectorCertificateError: Cannot connect to host api.dev.vm.validmind.ai:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')]